In [206]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [207]:
#for all that events need to find the closest player

In [208]:
project_dir = '/Users/volodymyr.kotov/Desktop/Kaggle/nfl_big_data_bowl_2025/data/nfl-big-data-bowl-2025'

In [209]:
games = pd.read_csv(f'{project_dir}/games.csv')
plays = pd.read_csv(f'{project_dir}/plays.csv')
players = pd.read_csv(f'{project_dir}/players.csv')

In [259]:
week = 3
tracking_df = pd.read_csv(f'{project_dir}/tracking_week_{week}.csv')
tracking_df['tracking_week'] = week

In [260]:
tracking_df_extended = tracking_df.copy()
tracking_df_extended['uid'] = tracking_df_extended.index

tracking_df_pass_events = tracking_df_extended[
    (tracking_df_extended['event'].isin(
                [
                    'pass_forward', 
                    'pass_arrived', 
                    'pass_outcome_incomplete', 
                    'pass_outcome_caught', 
                    'pass_outcome_interception', 
                    'qb_strip_sack'
                ]
            )
        ) 
        #& 
    #(tracking_df_extended['displayName'] == 'football')
]


In [261]:
#currently it will produce only a join to football player (a ball) and will show the event type

tracking_df_pass_events = pd.merge(
    tracking_df_pass_events,
    players,
    on=['nflId', 'displayName'],
    how='left'
)


#add a column is_qb  
tracking_df_pass_events['is_qb'] = np.where(
    tracking_df_pass_events['position'] == 'QB', True, False
)

In [262]:
fin_pass = tracking_df_pass_events[tracking_df_pass_events['event'] == 'pass_forward']

In [263]:
check = tracking_df_pass_events.groupby(['gameId', 'playId'])['is_qb'].nunique().reset_index()
data_check = check[check['is_qb'] == 1]

In [264]:
fin_pass = pd.merge(
    fin_pass,
    plays,
    on=['gameId', 'playId'],
    how='left'
)

In [265]:
def calculate_distances(group):
    team_a = group[group['possessionTeam'] == group['club']]
    team_b = group[group['possessionTeam'] != group['club']]
    
    position_QB = team_a[team_a['position'] == 'QB'][['x', 'y']].values[0]
    for idx, row in team_a.iterrows():
        group.at[idx, f"QB_distance_{row['position']}"] = np.sqrt((row['x'] - position_QB[0])**2 + (row['y'] - position_QB[1])**2)

    for idx, row in group.iterrows():
        if row['possessionTeam'] == row['club']:  
            opponent_distances = np.sqrt((team_b['x'] - row['x'])**2 + (team_b['y'] - row['y'])**2)
            group.at[idx, f"{row['position']}_closest_opponent_distance"] = opponent_distances.min()
    
    return group

group_columns = ['gameId', 'playId', 'x', 'y', 'club', 'possessionTeam', 'position']
df_result = fin_pass[
    (~fin_pass['gameId'].isin(data_check['gameId']))
    & ~(fin_pass['x'].isna())
    & ~(fin_pass['y'].isna())][group_columns].groupby(['gameId', 'playId']).apply(calculate_distances)

df_result.reset_index(drop=True, inplace=True)

/var/folders/mv/18wf18mn1c9bkwrp3fxhqr9c0000gp/T/ipykernel_53267/1004931420.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  & ~(fin_pass['y'].isna())][group_columns].groupby(['gameId', 'playId']).apply(calculate_distances)


In [266]:
end_pass = tracking_df_pass_events[tracking_df_pass_events['event'] != 'pass_forward']
end_pass = pd.merge(
    end_pass,
    plays,
    on=['gameId', 'playId'],
    how='left'
)

In [267]:
def get_closest_receiver(group):

    position_football = group[group['displayName'] == 'football'][['x', 'y']].values[0]

    for idx, row in group[~group['position'].isin(['QB', None])].iterrows():
        if row['possessionTeam'] == row['club'] and row['displayName'] != 'football':
            opponent_distances = group[
                (~group['position'].isin(['QB', None]))
                & (~group['position'].isna())].apply(
                lambda _row: {
                    'distance': np.sqrt((_row['x'] - position_football[0])**2 + (_row['y'] - position_football[1])**2),
                    'position': _row['position']
                },
                axis=1
            ).tolist()

            closest_opponent = min(opponent_distances, key=lambda x: x['distance'])
            
            group.at[idx, "football_closest_opponent_distance"] = closest_opponent['distance']
            group.at[idx, "football_closest_opponent_position"] = closest_opponent['position']

    return group
    
    

In [268]:
end_pass[end_pass['displayName'] == 'football']

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,...,yardsGained,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPointsAdded,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone
44,2022092600,57,NaN,football,157,AFTER_SNAP,2022-09-27 00:16:24.1,NaN,football,left,...,15,-0.015140,0.015140,1.016368,True,MAN,NaN,0,Cover-3,Zone
45,2022092600,57,NaN,football,160,AFTER_SNAP,2022-09-27 00:16:24.4,NaN,football,left,...,15,-0.015140,0.015140,1.016368,True,MAN,NaN,0,Cover-3,Zone
90,2022092600,160,NaN,football,203,AFTER_SNAP,2022-09-27 00:19:03.3,NaN,football,left,...,0,0.042287,-0.042287,-1.261845,True,UNDEFINED,NaN,0,Cover-3,Zone
91,2022092600,160,NaN,football,211,AFTER_SNAP,2022-09-27 00:19:04.1,NaN,football,left,...,0,0.042287,-0.042287,-1.261845,True,UNDEFINED,NaN,0,Cover-3,Zone
136,2022092600,182,NaN,football,228,AFTER_SNAP,2022-09-27 00:19:49.8,NaN,football,left,...,4,0.031340,-0.031340,-1.290083,True,NaN,NaN,0,2-Man,Man
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44596,2022092200,3858,NaN,football,128,AFTER_SNAP,2022-09-23 03:05:10.9,NaN,football,left,...,5,0.009796,-0.009796,0.059397,True,NaN,NaN,0,Quarters,Zone
44641,2022092200,3925,NaN,football,161,AFTER_SNAP,2022-09-23 03:08:07.4,NaN,football,left,...,0,0.030585,-0.030585,-1.184124,True,NaN,NaN,0,Quarters,Zone
44642,2022092200,3925,NaN,football,166,AFTER_SNAP,2022-09-23 03:08:07.9,NaN,football,left,...,0,0.030585,-0.030585,-1.184124,True,NaN,NaN,0,Quarters,Zone
44687,2022092200,4183,NaN,football,108,AFTER_SNAP,2022-09-23 03:24:58.2,NaN,football,left,...,-4,0.006967,-0.006967,-7.056592,True,NaN,NaN,0,Prevent,Other


In [269]:
group_columns = ['gameId', 'playId', 'x', 'y', 'club', 'possessionTeam', 'position', 'displayName']
df_football_result = end_pass[
    (~end_pass['gameId'].isin(data_check['gameId']))
    & ~(end_pass['x'].isna())
    & ~(end_pass['y'].isna())][group_columns].groupby(['gameId', 'playId']).apply(get_closest_receiver)

df_football_result.reset_index(drop=True, inplace=True)

/var/folders/mv/18wf18mn1c9bkwrp3fxhqr9c0000gp/T/ipykernel_53267/3924892220.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  & ~(end_pass['y'].isna())][group_columns].groupby(['gameId', 'playId']).apply(get_closest_receiver)


In [270]:
df_result.fillna(0, inplace=True)

In [271]:
lst = [col for col in df_result.columns if 'distance' in col]
lst.extend(['gameId', 'playId'])

In [272]:
final_result = df_result[lst].groupby(['gameId', 'playId']).max().reset_index()
final_result = pd.merge(
    final_result,
    plays,
    on=['gameId', 'playId'],
    how='left'
)

football_result = df_football_result[['gameId', 'playId', 'football_closest_opponent_position']].groupby(['gameId', 'playId']).first().reset_index()

final_result = pd.merge(
    final_result,
    football_result,
    on=['gameId', 'playId'],
    how='left'
)

In [273]:
final_result = pd.merge(
    final_result,
    games[games['week'] == week],
    on='gameId'
)

final_result['isHome'] = np.where(
    final_result['homeTeamAbbr'] == final_result['possessionTeam'],
    1, 0
)

final_result['pass_success'] = np.where(
    final_result['passResult'] == 'C', 1, 0
)

final_result_columns = [
    'gameId', 'playId', 'QB_distance_WR', 'QB_distance_G', 'QB_distance_T',
    'QB_distance_TE', 'QB_distance_C', 'QB_distance_QB', 'QB_distance_RB',
    'WR_closest_opponent_distance', 'G_closest_opponent_distance',
    'T_closest_opponent_distance', 'TE_closest_opponent_distance',
    'C_closest_opponent_distance', 'QB_closest_opponent_distance',
    'RB_closest_opponent_distance', 'QB_distance_FB',
    'FB_closest_opponent_distance', 'football_closest_opponent_position',
    'offenseFormation', 'possessionTeam', 'defensiveTeam', 'preSnapHomeTeamWinProbability',
    'preSnapHomeScore', 'preSnapVisitorScore', 'preSnapHomeTeamWinProbability', 'isHome', 'pass_success', 'week'
]

final_result['week'] = week


In [274]:
final_result[final_result_columns].to_csv(f'final_tracking_df_week_{week}.csv', index=False)

In [276]:
final_df = pd.DataFrame()

for week in range(1, 10):
    tracking_df = pd.read_csv(f'{project_dir}/tracking_week_{week}.csv')
    tracking_df['tracking_week'] = week
    tracking_df_extended = tracking_df.copy()
    tracking_df_extended['uid'] = tracking_df_extended.index

    tracking_df_pass_events = tracking_df_extended[
        (tracking_df_extended['event'].isin(
                    [
                        'pass_forward', 
                        'pass_arrived', 
                        'pass_outcome_incomplete', 
                        'pass_outcome_caught', 
                        'pass_outcome_interception', 
                        'qb_strip_sack'
                    ]
                )
            ) 
            #& 
        #(tracking_df_extended['displayName'] == 'football')
    ]
    
    #currently it will produce only a join to football player (a ball) and will show the event type

    tracking_df_pass_events = pd.merge(
        tracking_df_pass_events,
        players,
        on=['nflId', 'displayName'],
        how='left'
    )


    #add a column is_qb  
    tracking_df_pass_events['is_qb'] = np.where(
        tracking_df_pass_events['position'] == 'QB', True, False
    )

    fin_pass = tracking_df_pass_events[tracking_df_pass_events['event'] == 'pass_forward']

    check = tracking_df_pass_events.groupby(['gameId', 'playId'])['is_qb'].nunique().reset_index()
    data_check = check[check['is_qb'] == 1]

    fin_pass = pd.merge(
        fin_pass,
        plays,
        on=['gameId', 'playId'],
        how='left'
    )

    group_columns = ['gameId', 'playId', 'x', 'y', 'club', 'possessionTeam', 'position']
    df_result = fin_pass[
        (~fin_pass['gameId'].isin(data_check['gameId']))
        & ~(fin_pass['x'].isna())
        & ~(fin_pass['y'].isna())][group_columns].groupby(['gameId', 'playId']).apply(calculate_distances)

    df_result.reset_index(drop=True, inplace=True)

    end_pass = tracking_df_pass_events[tracking_df_pass_events['event'] != 'pass_forward']
    end_pass = pd.merge(
        end_pass,
        plays,
        on=['gameId', 'playId'],
        how='left'
    )

    group_columns = ['gameId', 'playId', 'x', 'y', 'club', 'possessionTeam', 'position', 'displayName']
    df_football_result = end_pass[
        (~end_pass['gameId'].isin(data_check['gameId']))
        & ~(end_pass['x'].isna())
        & ~(end_pass['y'].isna())][group_columns].groupby(['gameId', 'playId']).apply(get_closest_receiver)

    df_football_result.reset_index(drop=True, inplace=True)

    df_result.fillna(0, inplace=True)

    lst = [col for col in df_result.columns if 'distance' in col]
    lst.extend(['gameId', 'playId'])

    final_result = df_result[lst].groupby(['gameId', 'playId']).max().reset_index()
    final_result = pd.merge(
        final_result,
        plays,
        on=['gameId', 'playId'],
        how='left'
    )

    football_result = df_football_result[['gameId', 'playId', 'football_closest_opponent_position']].groupby(['gameId', 'playId']).first().reset_index()

    final_result = pd.merge(
        final_result,
        football_result,
        on=['gameId', 'playId'],
        how='left'
    )

    final_result = pd.merge(
        final_result,
        games[games['week'] == week],
        on='gameId'
    )

    final_result['isHome'] = np.where(
        final_result['homeTeamAbbr'] == final_result['possessionTeam'],
        1, 0
    )

    final_result['pass_success'] = np.where(
        final_result['passResult'] == 'C', 1, 0
    )

    final_result_columns = [
        'gameId', 'playId', 'QB_distance_WR', 'QB_distance_G', 'QB_distance_T',
        'QB_distance_TE', 'QB_distance_C', 'QB_distance_QB', 'QB_distance_RB',
        'WR_closest_opponent_distance', 'G_closest_opponent_distance',
        'T_closest_opponent_distance', 'TE_closest_opponent_distance',
        'C_closest_opponent_distance', 'QB_closest_opponent_distance',
        'RB_closest_opponent_distance', 'QB_distance_FB',
        'FB_closest_opponent_distance', 'football_closest_opponent_position',
        'offenseFormation', 'possessionTeam', 'defensiveTeam', 'preSnapHomeTeamWinProbability',
        'preSnapHomeScore', 'preSnapVisitorScore', 'preSnapHomeTeamWinProbability', 'isHome', 'pass_success', 'week'
    ]

    final_result['week'] = week

    final_df = pd.concat([final_df, final_result])
    print(f'week {week} is ready')


/var/folders/mv/18wf18mn1c9bkwrp3fxhqr9c0000gp/T/ipykernel_53267/323296307.py:56: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  & ~(fin_pass['y'].isna())][group_columns].groupby(['gameId', 'playId']).apply(calculate_distances)
/var/folders/mv/18wf18mn1c9bkwrp3fxhqr9c0000gp/T/ipykernel_53267/323296307.py:72: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  & ~(end_pass['y'].isna())][group_columns].groupby(['gam

week 1 is ready


/var/folders/mv/18wf18mn1c9bkwrp3fxhqr9c0000gp/T/ipykernel_53267/323296307.py:56: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  & ~(fin_pass['y'].isna())][group_columns].groupby(['gameId', 'playId']).apply(calculate_distances)
/var/folders/mv/18wf18mn1c9bkwrp3fxhqr9c0000gp/T/ipykernel_53267/323296307.py:72: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  & ~(end_pass['y'].isna())][group_columns].groupby(['gam

week 2 is ready


/var/folders/mv/18wf18mn1c9bkwrp3fxhqr9c0000gp/T/ipykernel_53267/323296307.py:56: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  & ~(fin_pass['y'].isna())][group_columns].groupby(['gameId', 'playId']).apply(calculate_distances)
/var/folders/mv/18wf18mn1c9bkwrp3fxhqr9c0000gp/T/ipykernel_53267/323296307.py:72: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  & ~(end_pass['y'].isna())][group_columns].groupby(['gam

...

/var/folders/mv/18wf18mn1c9bkwrp3fxhqr9c0000gp/T/ipykernel_53267/323296307.py:56: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  & ~(fin_pass['y'].isna())][group_columns].groupby(['gameId', 'playId']).apply(calculate_distances)
/var/folders/mv/18wf18mn1c9bkwrp3fxhqr9c0000gp/T/ipykernel_53267/323296307.py:72: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  & ~(end_pass['y'].isna())][group_columns].groupby(['gam

week 8 is ready


/var/folders/mv/18wf18mn1c9bkwrp3fxhqr9c0000gp/T/ipykernel_53267/323296307.py:56: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  & ~(fin_pass['y'].isna())][group_columns].groupby(['gameId', 'playId']).apply(calculate_distances)


week 9 is ready


/var/folders/mv/18wf18mn1c9bkwrp3fxhqr9c0000gp/T/ipykernel_53267/323296307.py:72: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  & ~(end_pass['y'].isna())][group_columns].groupby(['gameId', 'playId']).apply(get_closest_receiver)


In [277]:
final_df[final_result_columns].to_csv('final_df.csv', index=False)

### Preparing Dataset

### Training Model

In [322]:
df = pd.read_csv('final_df.csv')

In [323]:
df.drop(columns=['preSnapHomeTeamWinProbability.1', 'QB_distance_QB'], inplace=True)

In [324]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
cols_to_stand = [col for col in df.columns if 'distance' in col]
categorical_cols = [col for col in df.columns if 'distance' not in col]
df[cols_to_stand] = scaler.fit_transform(df[cols_to_stand])

In [325]:
import lightgbm as lgb

In [328]:
from sklearn.preprocessing import LabelEncoder


categorical_cols = ['football_closest_opponent_position', 'offenseFormation', 'possessionTeam', 'defensiveTeam']

le = LabelEncoder()

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le 

In [329]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['pass_success']), df['pass_success'], test_size = 0.3, random_state = 0)

In [336]:
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_cols)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data, categorical_feature=categorical_cols)

In [337]:
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 120,
    'learning_rate': 0.05,
}


num_round = 100
model = lgb.train(params, train_data, valid_sets=[test_data], num_boost_round=100)
                
#model_init.fit(train_data, y_train)

[LightGBM] [Info] Number of positive: 3775, number of negative: 2072
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000835 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4409
[LightGBM] [Info] Number of data points in the train set: 5847, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.645630 -> initscore=0.599886
[LightGBM] [Info] Start training from score 0.599886


In [338]:
from sklearn.metrics import precision_score, recall_score

y_pred = (model.predict(X_test) > 0.5).astype(int)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Precision: {precision}, Recall: {recall}")

Precision: 0.7621333333333333, Recall: 0.8903426791277259
